# 分類モデルを構築する: 美味しいアジア料理とインド料理


## 料理分類器 2

この2回目の分類レッスンでは、カテゴリカルデータを分類するための`さらなる方法`を探ります。また、どの分類器を選ぶかによる影響についても学びます。

### [**講義前のクイズ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **前提条件**

前のレッスンを完了していることを前提としています。なぜなら、以前学んだいくつかの概念を引き継ぐからです。

このレッスンでは、以下のパッケージが必要です：

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/)は、データサイエンスをより速く、簡単に、そして楽しくするために設計された[Rパッケージのコレクション](https://www.tidyverse.org/packages)です。

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/)フレームワークは、モデリングと機械学習のための[パッケージのコレクション](https://www.tidymodels.org/packages/)です。

-   `themis`: [themisパッケージ](https://themis.tidymodels.org/)は、不均衡データに対処するための追加のレシピステップを提供します。

以下のようにしてインストールできます：

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

または、以下のスクリプトを使用して、このモジュールを完了するために必要なパッケージがインストールされているか確認し、足りない場合は自動的にインストールすることもできます。


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. 分類マップ**

[前回のレッスン](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1)では、複数のモデルの中からどのように選ぶべきかという質問に取り組みました。選択は主にデータの特性や解決したい問題の種類（例えば、分類や回帰）に依存します。

以前、Microsoftのチートシートを使ってデータを分類する際のさまざまな選択肢について学びました。Pythonの機械学習フレームワークであるScikit-learnは、さらに詳細なチートシートを提供しており、分類器（別名エスティメーター）を絞り込む際に役立ちます。

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> ヒント: [このマップをオンラインで見る](https://scikit-learn.org/stable/tutorial/machine_learning_map/)と、ドキュメントを読むために道筋をクリックできます。
>
> [Tidymodelsのリファレンスサイト](https://www.tidymodels.org/find/parsnip/#models)も、さまざまなモデルタイプに関する優れたドキュメントを提供しています。

### **計画** 🗺️

このマップはデータをしっかり理解した後に非常に役立ちます。道筋をたどることで意思決定に至ることができます：

-   サンプルが50以上ある

-   カテゴリーを予測したい

-   ラベル付きデータがある

-   サンプル数が10万未満である

-   ✨ Linear SVCを選択できる

-   それがうまくいかない場合、数値データがあるので

    -   ✨ KNeighbors Classifierを試すことができる

        -   それでもうまくいかない場合は、✨ SVCや✨ Ensemble Classifiersを試してみる

この道筋は非常に役立ちます。それでは、[tidymodels](https://www.tidymodels.org/)モデリングフレームワークを使って始めましょう。これは、良い統計的実践を促進するために開発された、Rパッケージの一貫性があり柔軟なコレクションです 😊。

## 2. データを分割し、不均衡なデータセットに対処する

前回のレッスンで、私たちの料理ジャンル間で共通する要素があることを学びました。また、料理ジャンルの数にかなりの偏りがあることも分かりました。

これらに対処するために以下を行います：

-   異なる料理ジャンル間で混乱を引き起こす最も一般的な要素を`dplyr::select()`を使って削除します。

-   データを前処理してモデリングの準備をするために、`recipe`を使用し、`オーバーサンプリング`アルゴリズムを適用します。

これらは前回のレッスンで既に見た内容なので、簡単に進められるはずです 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### 不均衡データへの対処

不均衡データは、モデルの性能に悪影響を及ぼすことがよくあります。多くのモデルは観測数が均等である場合に最も良いパフォーマンスを発揮し、不均衡なデータでは苦戦する傾向があります。

不均衡データセットに対処する方法は主に2つあります：

- 少数派クラスに観測値を追加する: `オーバーサンプリング` 例えば、SMOTEアルゴリズムを使用して、少数派クラスの新しい例をこれらのケースの近傍を基に合成的に生成する方法。

- 多数派クラスから観測値を削除する: `アンダーサンプリング`

前回のレッスンでは、`recipe` を使用して不均衡データセットに対処する方法を示しました。`recipe` は、データセットをデータ分析の準備が整うようにするためにどのステップを適用すべきかを記述した設計図のようなものと考えることができます。今回の場合、`training set` における料理の数を均等に分布させたいと考えています。それでは、始めましょう。


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

さあ、モデルのトレーニングを始めましょう 👩‍💻👨‍💻！

## 3. 多項ロジスティック回帰モデルを超えて

前回のレッスンでは、多項ロジスティック回帰モデルについて学びました。今回は、分類のためのより柔軟なモデルを探っていきましょう。

### サポートベクターマシン

分類の文脈において、`Support Vector Machines`（サポートベクターマシン）は、クラスを「最適に」分離する*ハイパープレーン*を見つけようとする機械学習手法です。簡単な例を見てみましょう：

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~はクラスを分けません。H2~は分けますが、余白は小さいです。H3~は最大の余白でクラスを分けます。

#### 線形サポートベクター分類器

サポートベクタークラスタリング（SVC）は、機械学習技術の一つであるサポートベクターマシン（SVM）ファミリーに属します。SVCでは、ハイパープレーンはトレーニングデータの`ほとんど`を正しく分離するように選ばれますが、`一部の観測値を誤分類する可能性`があります。一部のポイントが誤った側に位置することを許容することで、SVMは外れ値に対してより頑健になり、新しいデータへの一般化能力が向上します。この違反を調整するパラメータは`cost`と呼ばれ、デフォルト値は1です（`help("svm_poly")`を参照してください）。

ポリノミアルSVMモデルで`degree = 1`を設定することで、線形SVCを作成してみましょう。


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

ワークフローに前処理ステップとモデル仕様をまとめたので、線形SVCをトレーニングし、その結果を評価してみましょう。パフォーマンス指標として、以下を評価するメトリックセットを作成します：`accuracy`、`sensitivity`、`Positive Predicted Value`、および `F Measure`。

> `augment()` は、指定されたデータに予測結果の列を追加します。


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### サポートベクターマシン

サポートベクターマシン（SVM）は、クラス間の非線形な境界を扱うためにサポートベクトル分類器を拡張したものです。本質的に、SVMは*カーネルトリック*を使用して特徴空間を拡大し、クラス間の非線形な関係に適応します。SVMで使用される一般的で非常に柔軟なカーネル関数の1つが*放射基底関数*です。それが私たちのデータでどのように機能するか見てみましょう。


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

ずっと良いですね 🤩！

> ✅ こちらもご覧ください:
>
> -   [*サポートベクターマシン*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*サポートベクターマシン*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> さらなる学習のために。

### 最近傍法分類器

*K*-最近傍法 (KNN) は、各観測値を他の観測値との「類似性」に基づいて予測するアルゴリズムです。

これをデータに適用してみましょう。


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

このモデルはあまり良いパフォーマンスを発揮していないようです。おそらく、`help("nearest_neighbor")`を参照してモデルの引数を変更すれば、パフォーマンスが向上する可能性があります。ぜひ試してみてください。

> ✅ 以下を参照してください:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> *K*-Nearest Neighbors分類器について詳しく学ぶにはこちらをご覧ください。

### アンサンブル分類器

アンサンブルアルゴリズムは、複数の基本推定器を組み合わせることで最適なモデルを作成します。その方法は以下のいずれかです:

`bagging`: 基本モデルの集合に対して*平均化関数*を適用する

`boosting`: 一連のモデルを順次構築し、それぞれが前のモデルを改善することで予測性能を向上させる

まずは、ランダムフォレストモデルを試してみましょう。このモデルは、多数の決定木を構築し、それに平均化関数を適用することで、全体としてより優れたモデルを作成します。


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

よくやった 👏！

では、Boosted Treeモデルを試してみましょう。

Boosted Treeは、逐次的な決定木のシリーズを作成するアンサンブル手法を定義します。このシリーズでは、各木が前の木の結果に依存し、誤差を段階的に減らそうとします。この手法は、誤分類された項目の重み付けに焦点を当て、次の分類器の適合を調整して修正を試みます。

このモデルを適合させる方法はいくつかあります（`help("boost_tree")`を参照してください）。この例では、`xgboost`エンジンを使用してBoosted Treeを適合させます。


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ ぜひ以下をご覧ください：
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - AdaBoostモデルについて解説しており、xgboostの良い代替案となります。
>
> アンサンブル分類器についてさらに学ぶために。

## 4. 追加 - 複数モデルの比較

このラボでは、かなり多くのモデルを適用しました 🙌。異なる前処理やモデル仕様のセットから多くのワークフローを作成し、それぞれの性能指標を一つずつ計算するのは、面倒で手間がかかることがあります。

これを解決するために、トレーニングセットに対して複数のワークフローを適用し、テストセットに基づいて性能指標を返す関数を作成してみましょう。[purrr](https://purrr.tidyverse.org/)パッケージの`map()`と`map_dfr()`を使用して、リスト内の各要素に関数を適用します。

> [`map()`](https://purrr.tidyverse.org/reference/map.html)関数を使用すると、多くのforループをより簡潔で読みやすいコードに置き換えることができます。[`map()`](https://purrr.tidyverse.org/reference/map.html)関数について学ぶ最適な場所は、R for Data Scienceの[反復処理の章](http://r4ds.had.co.nz/iteration.html)です。


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) パッケージは、ユーザーが多数のモデルを作成し簡単に適合させることを可能にしますが、主に `クロスバリデーション` のようなリサンプリング技術と連携するよう設計されています。このアプローチについてはまだ学んでいません。

## **🚀チャレンジ**

これらの技術には多くのパラメータがあり、例えば SVM の `cost`、KNN の `neighbors`、ランダムフォレストの `mtry`（ランダムに選択された予測子）などを調整することができます。

各モデルのデフォルトパラメータを調べ、それらのパラメータを調整することでモデルの品質にどのような影響があるかを考えてみましょう。

特定のモデルとそのパラメータについて詳しく知りたい場合は、次のコマンドを使用してください: `help("model")` 例: `help("rand_forest")`

> 実際には、これらの *最適な値* を *推定* するために、多くのモデルを `シミュレーションデータセット` でトレーニングし、それらのモデルがどれだけうまく機能するかを測定します。このプロセスは **チューニング** と呼ばれます。

### [**講義後のクイズ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **復習と自己学習**

これらのレッスンには専門用語が多く含まれているため、[このリスト](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) を確認して役立つ用語を復習する時間を取ってください！

#### 感謝の言葉:

[`Allison Horst`](https://twitter.com/allison_horst/) に感謝します。彼女は R をより親しみやすく魅力的にする素晴らしいイラストを作成してくれました。彼女の [ギャラリー](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) でさらに多くのイラストを見ることができます。

[Cassie Breviu](https://www.twitter.com/cassieview) と [Jen Looper](https://www.twitter.com/jenlooper) に感謝します。このモジュールの元となる Python バージョンを作成してくれました ♥️

楽しく学びましょう！

[Eric](https://twitter.com/ericntay)、Gold Microsoft Learn Student Ambassador

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>@allison_horst によるアートワーク</figcaption>



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を期すよう努めておりますが、自動翻訳には誤りや不正確な表現が含まれる可能性があります。元の言語で記載された原文を公式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用に起因する誤解や誤認について、当社は一切の責任を負いません。
